In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# load data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### BASIC MNIST ML

In [53]:
# each image has 28x28=784 pixels, None means unrestricted number of images
x = tf.placeholder(tf.float32, [None, 784])

# 10 means 10 different out put classes
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

In [62]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [63]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 100 == 0:
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print("Iteration={0}, train accuracy={1}, test acurracy={2}".format(
            i, train_accuracy, test_accuracy))

Iteration=0, train accuracy=0.5299999713897705, test acurracy=0.3165999948978424
Iteration=100, train accuracy=0.9399999976158142, test acurracy=0.8974000215530396
Iteration=200, train accuracy=0.8999999761581421, test acurracy=0.9036999940872192
Iteration=300, train accuracy=0.9399999976158142, test acurracy=0.9097999930381775
Iteration=400, train accuracy=0.9300000071525574, test acurracy=0.9128000140190125
Iteration=500, train accuracy=0.9399999976158142, test acurracy=0.9136000275611877
Iteration=600, train accuracy=0.9399999976158142, test acurracy=0.9156000018119812
Iteration=700, train accuracy=0.9700000286102295, test acurracy=0.916100025177002
Iteration=800, train accuracy=0.9300000071525574, test acurracy=0.9193999767303467
Iteration=900, train accuracy=0.949999988079071, test acurracy=0.9165999889373779


In [64]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9151


### MNIST CNN

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [5]:
# first convolution layer
x = tf.placeholder(tf.float32, [None, 784])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [6]:
# second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [7]:
# fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [8]:
# dropout at fully connected layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [9]:
# final layer for output
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [11]:
# loss and accuracy
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 500 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step {0}, training accuracy {1}'.format(i, train_accuracy))
            print('test accuracy %g' % accuracy.eval(feed_dict={
                x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.019999999552965164
test accuracy 0.0818
step 500, training accuracy 0.9200000166893005
test accuracy 0.9449
step 1000, training accuracy 0.9399999976158142
test accuracy 0.9617
step 1500, training accuracy 0.9800000190734863
test accuracy 0.9702
step 2000, training accuracy 0.9599999785423279
test accuracy 0.9723
step 2500, training accuracy 1.0
test accuracy 0.9791
step 3000, training accuracy 0.9800000190734863
test accuracy 0.98
step 3500, training accuracy 0.9800000190734863
test accuracy 0.9828
step 4000, training accuracy 1.0
test accuracy 0.9833
step 4500, training accuracy 0.9800000190734863
test accuracy 0.9849
step 5000, training accuracy 1.0
test accuracy 0.9843
step 5500, training accuracy 1.0
test accuracy 0.9849
step 6000, training accuracy 1.0
test accuracy 0.9874
step 6500, training accuracy 0.9800000190734863
test accuracy 0.9884
step 7000, training accuracy 1.0
test accuracy 0.9895
step 7500, training accuracy 1.0
test accuracy 0.989
step 8